Empaquetado de modelos con MLflow
=================================

## ¿Que es MLflow?

MLflow es un framework de aprendizaje automático que permite manejar multiples aspectos del ciclo de vida de desarrollo de modelos. Puntualmente provee capacidades para:

1. Realizar tracking de experimentos, conocido como **MLflow tracking**.
1. Administrar modelos, conocido como **MLflow Model Registry**.
1. Desplegar modelos, conocido como **MLflow Models**

Anteriormente en este curso utilizamos *Comet* para realizar las acividades 1) y 2). Ambos estandares permiten realizar estas operaciones de una forma similar. Sin embargo, MLflow, ademas de ser un estandard de código abierto, permite desplegar los modelos y ejecutarlos en producción.

En este ejemplo, veremos como utilizar MLflow para estas operaciones:

## Entrenando y optimizando parámetros para el problema censo de la UCI

### Instalación

In [93]:
%pip install mlflow scikit-learn pandas numpy -q

### Sobre el conjunto de datos del censo UCI

El conjunto de datos del censo de la UCI es un conjunto de datos en el que cada registro representa a una persona. Cada registro contiene 14 columnas que describen a una una sola persona, de la base de datos del censo de Estados Unidos de 1994. Esto incluye información como la edad, el estado civil y el nivel educativo. La tarea es determinar si una persona tiene un ingreso alto (definido como ganar más de $50 mil al año). Esta tarea, dado el tipo de datos que utiliza, se usa a menudo en el estudio de equidad, en parte debido a los atributos comprensibles del conjunto de datos, incluidos algunos que contienen tipos sensibles como la edad y el género, y en parte también porque comprende una tarea claramente del mundo real.

Descargamos el conjunto de datos

In [1]:
!wget https://santiagxf.blob.core.windows.net/public/datasets/uci_census.zip \
    --quiet --no-clobber
!mkdir -p datasets/uci_census
!unzip -qq uci_census.zip -d datasets/uci_census

Lo importamos

In [18]:
import pandas as pd
import numpy as np

train = pd.read_csv('datasets/uci_census/data/adult-train.csv')
test = pd.read_csv('datasets/uci_census/data/adult-test.csv')

### Creando un modelo utilizando pipelines

Preparando nuestros conjuntos de datos

In [45]:
X_train = train.drop(['income'], axis=1)
y_train = train['income'].to_numpy()
X_test = test.drop(['income'], axis=1)
y_test = test['income'].to_numpy()

In [46]:
classes = train['income'].unique().tolist()
features = X_train.columns.values.tolist()
categorical_features = X_train.dtypes[X_train.dtypes == 'object'].index.tolist()

Realizaremos un pequeño preprocesamiento antes de entrenar el modelo:

- Imputaremos los valores faltantes de las caracteristicas numéricas con la media
- Imputaremos los valores faltantes de las caracteristicas categóricas con el valor `?`
- Escalaremos los valores numericos utilizando un `StandardScaler`
- Codificaremos las variables categóricas utilizando `OneHotEncoder`

In [47]:
from typing import Tuple, List

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


pipe_cfg = {
    'num_cols': X_train.dtypes[X_train.dtypes == 'int64'].index.values.tolist(),
    'cat_cols': X_train.dtypes[X_train.dtypes == 'object'].index.values.tolist(),
}

num_pipe = Pipeline([
    ('num_imputer', SimpleImputer(strategy='median')),
    ('num_scaler', StandardScaler())
])

cat_pipe = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

transformations = ColumnTransformer([
    ('num_pipe', num_pipe, pipe_cfg['num_cols']),
    ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
])

Nuestro modelo estará basado en un `GradientBoostingClassifier`:

In [56]:
parameters = {
    'subsample': 0.7,
    'learning_rate': 0.07,
    'max_depth': 7,
    'n_estimators': 200,
}

In [57]:
from sklearn.ensemble import GradientBoostingClassifier

base_model = GradientBoostingClassifier(**parameters)

> **Tip:** Si no conocía el operador `**` en Python, el mismo tiene 2 significados: El primero, la operación exponencial (como en `2**4`). El segundo, cuando el operador está en frente de un diccionario, "desempaca" los valores de un diccionario y los pasa como argumentos a una función. En este emplo, los diferentes valores del dicionario están siendo utilizados como argumentos para contruir el `XGBClassifier`.

En esta ocacion, en lugar de ejecutar el preprocesamiento sobre el conjunto de datos y luego enviarlo al modelo, construiremos un nuevo pipeline que contendrá los 2 pasos. La librería Scikit-learn le permite realizar estas combinaciones:

In [58]:
model_pipeline = Pipeline([
    ('preprocessing', transformations),
    ('classifier', base_model),
])

Entrenemos el pipeline completo. Note que en esta acción, todos los pasos del pipeline serán ajustados (fit). Esto quiere decir que cualquier valor que se requiera aprender para ejecutar las transformaciones sucederá en este paso. También el modelo se ajustará y estimará sus parámetros:

In [59]:
model = model_pipeline.fit(X_train, y_train)

Note como el pipeline completo tiene el método predict para retornar predicciones. Sin embargo, este metodo ahora toma como entrada predictores sin procesar directamente. El mismo pipeline se encarga de preprocesarlos:

In [60]:
predictions = model.predict(X_test)

Podemos revisar la performance del modelo:

In [61]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       <=50K       0.90      0.94      0.92     12435
        >50K       0.77      0.66      0.71      3846

    accuracy                           0.87     16281
   macro avg       0.83      0.80      0.81     16281
weighted avg       0.87      0.87      0.87     16281



### Empaquetando el modelo con MLflow

Empaquetaremos el modelo que construimos con MLflow. MLflow dispone de un estandar para realizar este empaquetado, MLmodel, el cual dispone de un conjunto de metadata que hace más sencillo su despliegue luego:

In [82]:
import mlflow
from mlflow.models.signature import infer_signature

In [89]:
mlflow.sklearn.save_model(
    sk_model=model,
    path="model",
    signature=infer_signature(X_test, predictions)
)

/usr/local/lib/python3.10/dist-packages/mlflow/models/signature.py:152: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.


> El método `infer_signature` le permite a MLflow identificar cuales son los predictores de entrada que el modelo necesita y cuales son las predicciones que genera:

In [90]:
!ls model

conda.yaml  MLmodel  model.pkl	python_env.yaml  requirements.txt


Podemos inspeccionar como luce la metadata de este paquete:

In [91]:
!cat model/MLmodel

flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.10.12
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.5.0
model_uuid: b009b3261e634572ac91994c4814e6f6
signature:
  inputs: '[{"type": "long", "name": "age"}, {"type": "string", "name": "workclass"},
    {"type": "long", "name": "fnlwgt"}, {"type": "string", "name": "education"}, {"type":
    "long", "name": "education-num"}, {"type": "string", "name": "marital-status"},
    {"type": "string", "name": "occupation"}, {"type": "string", "name": "relationship"},
    {"type": "string", "name": "race"}, {"type": "string", "name": "gender"}, {"type":
    "long", "name": "capital-gain"}, {"type": "long", "name": "capital-loss"}, {"type":
    "long", "name": "hours-per-week"}, {"type": "string", "name": "nat